- LeNet-5
  - A CNN model proposed by Yann LeCun in 1998
  - Be designed for handwritten digit recognition, especially for the MNIST dataset
  - Structure
    - Input layer
    - Convolutional layers: 3
    - Pooling layers: 2
    - Fully connected layers: 2
    - Output layer

### Preparation

In [1]:
# Import modules
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

### Load Data: CIFAR-10

In [2]:
# Set transforms
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomVerticalFlip(p = 0.3),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2))
])

In [3]:
# Get data and loaders
train_data = torchvision.datasets.CIFAR10(root = './data/0704-LeNet-5',
                                          train = True,
                                          download = True, 
                                          transform = train_transform)
train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size = 64,
                                           shuffle = True)

test_data = torchvision.datasets.CIFAR10(root = './data/0704-LeNet-5',
                                         train = False,
                                         download = True,
                                         transform = test_transform)
test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size = 64,
                                          shuffle = False)

Files already downloaded and verified
Files already downloaded and verified


### Create a LeNet-5 Model

In [4]:
# Define a LeNet-5 class using Receptive Field
class LeNet(nn.Module):

    # Initialize a model
    def __init__ (self):
        # Initialize class itself
        super(LeNet, self).__init__()
        
        # Set Convolutional layers and Max-Pooling layers
        self.conv1 = nn.Conv2d(3, 32, 3)  # Input channel: 3, Output channel: 32, Kernal size: 3x3
        self.pool = nn.MaxPool2d(2, 2)    # MaxPolling layer with a kernel size of 2x2, stride of 2
        self.conv2 = nn.Conv2d(32, 64, 3) # Input channel: 32, Output channel: 64, Kernal size: 3x3
        
        # Set Fully connected layers
        self.fc1 = nn.Linear(64 * 6 * 6, 64) # Input size: 64 * 6 * 6, Output size: 64
        self.fc2 = nn.Linear(64, 10)         # Input size: 64, Output size(Class): 10
        
    # Define forward
    def forward(self, x):
        
        # Pass 'x' through: the convolutional layers, ReLu activation, max-pooling layers
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        
        # Output is flattened into Rank1 tensor
        x = torch.flatten(x, 1)
        
        # Pass the falttened layer to the first fully connected layer with ReLu activation function
        x = torch.relu(self.fc1(x))
        
        # Pass the result to the second fully connected layer
        x = self.fc2(x)
        
        return x

### Fit Model 

In [5]:
# Define a model
def train_and_eval(model):
    # Set Loss Function
    criterion = nn.CrossEntropyLoss()
    
    # Set optimizer
    optimizer = optim.SGD(model.parameters(),
                          lr = 0.001,
                          momentum = 0.9)
    
    
    # Set a learning loop
    for epoch in range(5):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            
            inputs, labels = data
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 200 == 190:
                print('[%d,  %gd] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
                running_loss = 0.0
    
    print('Training completed!')
    
    
    # Evaluate models
    # Initialize parameters
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            
            inputs, labels = data
            outputs = model(inputs)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = 100 * correct / total
    print('Accuracy: %.2f %%' % accuracy)

### Run Models

In [6]:
# Run model
print('LeNet-5: ')
model = LeNet()
train_and_eval(model)

# Compare parameters
print('Num of parameters: ')
print('LeNet-5', sum(p.numel() for p in model.parameters()))

LeNet-5: 
[1,  191d] loss: 2.139
[1,  391d] loss: 2.025
[1,  591d] loss: 1.860
[2,  191d] loss: 1.589
[2,  391d] loss: 1.610
[2,  591d] loss: 1.564
[3,  191d] loss: 1.446
[3,  391d] loss: 1.475
[3,  591d] loss: 1.454
[4,  191d] loss: 1.362
[4,  391d] loss: 1.392
[4,  591d] loss: 1.369
[5,  191d] loss: 1.283
[5,  391d] loss: 1.304
[5,  591d] loss: 1.307
Training completed!
Accuracy: 56.76 %
Num of parameters: 
LeNet-5 167562
